In [67]:
import os
import pandas as pd
path_, filename_, category_, article_or_summary_ = [],[],[],[]
for dirname, _, filenames in os.walk('/content/Untitled Folder'):
    for filename in filenames:
        path_.append(os.path.join(dirname, filename))
        filename_.append(filename)
        category_.append(dirname.split("/")[-1])
        article_or_summary_.append(dirname.split("/")[-2])

In [68]:
df = pd.DataFrame({"path":path_, "filename":filename_, "category":category_, "article_or_summary":article_or_summary_}, columns=["path", "filename", "category", "article_or_summary"])
df

,path,filename,category,article_or_summary
0,/content/Untitled Folder/e.txt,e.txt,Untitled Folder,content
1,/content/Untitled Folder/e1.txt,e1.txt,Untitled Folder,content


In [ ]:
!pip install cufflinks
!pip install plotly-express
import plotly_express as pe
import cufflinks as cf

cf.go_offline()

In [ ]:
from collections import Counter

ct = Counter(df[df['article_or_summary']=="News Articles"]["category"])
pd.DataFrame({"category":ct.keys(), "value":ct.values()}).iplot(kind='bar', x='category', y='value')

In [46]:
print(g)

None


In [ ]:
pd.DataFrame({"category":ct.keys(), "value":ct.values()}).iplot(kind='box')

In [ ]:
pd.DataFrame({"category":ct.keys(), "value":ct.values()}).iplot(kind='bubble', x='category', y='value', size='value')

In [ ]:
g = pd.DataFrame({"category":ct.keys(), "value":ct.values()}).iplot(kind='pie', labels="category", values='value')

In [52]:
print(g)

None


In [64]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
import numpy as np
import networkx as nx
import re

In [65]:
def read_article(text):
    sentences =[]
    sentences = sent_tokenize(text)
    for sentence in sentences:
        sentence.replace("[^a-zA-Z0-9]"," ")
    return sentences

In [74]:
#file_path = df[df['article_or_summary']=='News Articles'].iloc[0]['path']
with open('/content/Untitled Folder/e.txt', "r") as f:
    article = f.read()

In [75]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [76]:
sent_tok = read_article(article)
sent_tok

['e Officer-in-Charge\nof G. Ghati P.S.',
 'and lodged a written complaint alleging that his sister Belarani\nGhosh was given marriage with Arun Ghosh of Pabradihi village on 18th Falgoon,\n1405 B.S.',
 'according to Hindu rites and customs.',
 'After some days of her marriage she\nwas physically and mentally tortured by her husband, father-in-law and mother-in\x02law and for that she occasionally came to his house being unable to bear such\ntorture.',
 "The de facto complainant persuaded her and sent her to her father-in-law's\nhouse.",
 'On 13th November, 2001 at about 4 p.m. two persons namely, Bhajahari\nMondal and Uttam Mondal of village Papradihi reported to him in their house that his\nsister was physically tortured by her husband in their presence and she had died.',
 'They saw Arun Ghosh to assault his sister while returning from the paddy field.',
 'On\ngetting this information he and his para people visited to the house of the accused at\nnight on that day and found his sist

In [ ]:
from textblob import TextBlob
mod_sent = []
for tok in sent_tok:
    blob_obj = TextBlob(tok)
    correct_sent = str(blob_obj.correct())
    print(f"\033[94m Original Token : {tok} \033[0m")
    print(f"\033[92m Corrected Token: {correct_sent} \033[92m")
    mod_sent.append(correct_sent)

In [78]:
" ".join(mod_sent)

"e Officer-in-Charge\nof G. That P.S. and lodged a written complaint alleging that his sister Belarani\nThose was given marriage with Run Those of Pabradihi village on with Balloon,\n1405 B.S. according to Hindu rates and customs. After some days of her marriage she\nwas physically and mentally tortured by her husband, father-in-law and mother-in\x02law and for that she occasionally came to his house being unable to bear such\ntorture. The de facto complaint persuaded her and sent her to her father-in-law's\nhouse. In with November, 2001 at about 4 p.m. two persons namely, Bhajahari\nMonday and Steam Monday of village Papradihi reported to him in their house that his\nsister was physically tortured by her husband in their presence and she had died. They saw Run Those to assault his sister while returning from the daddy field. In\ngetting this information he and his para people visited to the house of the accused at\nnight on that day and found his sister lying dead in the cowshed of th

In [79]:
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

def sentence_similarity(sent1,sent2,embed):
    A = embed([sent1])[0]
    B = embed([sent2])[0]
    return 1 - (np.dot(A,B)/(np.linalg.norm(A)*np.linalg.norm(B)))

In [ ]:
print(f"\033[92m Sentence 1 : {mod_sent[0]}")
print(f"\033[92m Sentence 2 : {mod_sent[1]}")
print(f"\033[92m Similarity Score : {sentence_similarity(mod_sent[0], mod_sent[1], embed)}")

In [95]:
def build_similarity_matrix(sentences,embeds):
    similarity_matrix = np.zeros((len(sentences),len(sentences)))
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1!=idx2:
                similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1],sentences[idx2],embeds)
    return similarity_matrix

In [82]:
sim_mat = build_similarity_matrix(mod_sent, embed)

In [96]:
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
import networkx
from bokeh.io import output_notebook, show, save

output_notebook()

g = nx.Graph()

for i in range(sim_mat.shape[0]):
    for j in range(sim_mat.shape[1]):
        if sim_mat[i][j] >=.9:
            g.add_edge(i, j)

HOVER_TOOLTIPS = [("sent_tok", "@index")]
plot = figure(tooltips = HOVER_TOOLTIPS, tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1))

network_graph = from_networkx(g, networkx.spring_layout, scale=7, center=(0, 0))
network_graph.node_renderer.glyph = Circle(size=15,fill_color='green')
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)
plot.renderers.append(network_graph)
show(plot)

In [97]:
#file_path_summary = df[df['article_or_summary']=='Summaries'].iloc[0]['path']
with open('/content/Untitled Folder/e.txt', "r") as f:
    actual_summary = f.read()

In [86]:
def generate_summary(text,top_n,embeds):
    summarize_text = []
    sentences = read_article(text)
    sentence_similarity_matrix = build_similarity_matrix(sentences,embeds)
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
    scores = nx.pagerank(sentence_similarity_graph)
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)),reverse=True)
    for i in range(top_n):
        summarize_text.append(ranked_sentences[i][1])
    return " ".join(summarize_text)

In [87]:
Original_Text = " ".join(mod_sent)
Summarized_Text = generate_summary(Original_Text, top_n=5, embeds=embed)

In [88]:
Original_Text

"e Officer-in-Charge\nof G. That P.S. and lodged a written complaint alleging that his sister Belarani\nThose was given marriage with Run Those of Pabradihi village on with Balloon,\n1405 B.S. according to Hindu rates and customs. After some days of her marriage she\nwas physically and mentally tortured by her husband, father-in-law and mother-in\x02law and for that she occasionally came to his house being unable to bear such\ntorture. The de facto complaint persuaded her and sent her to her father-in-law's\nhouse. In with November, 2001 at about 4 p.m. two persons namely, Bhajahari\nMonday and Steam Monday of village Papradihi reported to him in their house that his\nsister was physically tortured by her husband in their presence and she had died. They saw Run Those to assault his sister while returning from the daddy field. In\ngetting this information he and his para people visited to the house of the accused at\nnight on that day and found his sister lying dead in the cowshed of th

In [89]:
Summarized_Text

"e Officer-in-Charge\nof G. That P.S. according to Hindu rates and customs. They saw Run Those to assault his sister while returning from the daddy field. The de facto complaint persuaded her and sent her to her father-in-law's\nhouse. After some days of her marriage she\nwas physically and mentally tortured by her husband, father-in-law and mother-in\x02law and for that she occasionally came to his house being unable to bear such\ntorture."

In [90]:
actual_summary

"e Officer-in-Charge\nof G. Ghati P.S. and lodged a written complaint alleging that his sister Belarani\nGhosh was given marriage with Arun Ghosh of Pabradihi village on 18th Falgoon,\n1405 B.S. according to Hindu rites and customs. After some days of her marriage she\nwas physically and mentally tortured by her husband, father-in-law and mother-in\x02law and for that she occasionally came to his house being unable to bear such\ntorture. The de facto complainant persuaded her and sent her to her father-in-law's\nhouse. On 13th November, 2001 at about 4 p.m. two persons namely, Bhajahari\nMondal and Uttam Mondal of village Papradihi reported to him in their house that his\nsister was physically tortured by her husband in their presence and she had died.\nThey saw Arun Ghosh to assault his sister while returning from the paddy field. On\ngetting this information he and his para people visited to the house of the accused at\nnight on that day and found his sister lying dead in the cowshed

In [91]:
import nltk

hypothesis = Summarized_Text
reference = actual_summary
BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis)
print(f"BLEUscore : {BLEUscore}")

BLEUscore : 0.18289556294059692


In [92]:
print(f"Senetence Similarity Score : {sentence_similarity(Summarized_Text, actual_summary, embed)}")

Senetence Similarity Score : 0.264262318611145


In [93]:
!pip install  sumy
import sumy
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

In [106]:
# For Strings
parser = PlaintextParser.from_string(Original_Text,Tokenizer("english"))

summarizer = LexRankSummarizer()
#Summarize the document with 2 sentences
summary = summarizer(parser.document, 5)
#print(summary)
for sentence in summary:
    print(sentence)

e Officer-in-Charge of G. That P.S.
and lodged a written complaint alleging that his sister Belarani Those was given marriage with Run Those of Pabradihi village on with Balloon, 1405 B.S.
After some days of her marriage she was physically and mentally tortured by her husband, father-in-law and mother-inlaw and for that she occasionally came to his house being unable to bear such torture.
In with November, 2001 at about 4 p.m. two persons namely, Bhajahari Monday and Steam Monday of village Papradihi reported to him in their house that his sister was physically tortured by her husband in their presence and she had died.
In that night he came to the police station from the house of the accused and on the next morning he lodged a written complaint at G. That P.S.


In [107]:
# Assuming Original_Text contains your sentence
parser = PlaintextParser.from_string(Original_Text, Tokenizer("english"))

summarizer = LexRankSummarizer()
# Summarize the document with 5 sentences
summary = summarizer(parser.document, 5)

# Store the summarized sentences in a variable
summarized_text = "\n".join(str(sentence) for sentence in summary)

# Now you can use 'summarized_text' elsewhere in your code
print(summarized_text)


e Officer-in-Charge of G. That P.S.
and lodged a written complaint alleging that his sister Belarani Those was given marriage with Run Those of Pabradihi village on with Balloon, 1405 B.S.
After some days of her marriage she was physically and mentally tortured by her husband, father-in-law and mother-inlaw and for that she occasionally came to his house being unable to bear such torture.
In with November, 2001 at about 4 p.m. two persons namely, Bhajahari Monday and Steam Monday of village Papradihi reported to him in their house that his sister was physically tortured by her husband in their presence and she had died.
In that night he came to the police station from the house of the accused and on the next morning he lodged a written complaint at G. That P.S.
